# Courses Demo
This Jupyter notebook is for exploring the data set courses20-21.json
which consists of all Brandeis courses in the 20-21 academic year (Fall20, Spr21, Sum21) 
which had at least 1 student enrolled.

First we need to read the json file into a list of Python dictionaries

In [1]:
import json

In [2]:
with open("courses20-21.json","r",encoding='utf-8') as jsonfile:
    courses = json.load(jsonfile)

## Structure of a course
Next we look at the fields of each course dictionary and their values

In [3]:
print('there are',len(courses),'courses in the dataset')
print('here is the data for course 1246')
courses[1246]

there are 7813 courses in the dataset
here is the data for course 1246


{'limit': 28,
 'times': [{'start': 1080, 'end': 1170, 'days': ['w', 'm']}],
 'enrolled': 4,
 'details': 'Instruction for this course will be offered remotely. Meeting times for this course are listed in the schedule of classes (in ET).',
 'type': 'section',
 'status_text': 'Open',
 'section': '1',
 'waiting': 0,
 'instructor': ['An', 'Huang', 'anhuang@brandeis.edu'],
 'coinstructors': [],
 'code': ['MATH', '223A'],
 'subject': 'MATH',
 'coursenum': '223A',
 'name': 'Lie Algebras: Representation Theory',
 'independent_study': False,
 'term': '1203',
 'description': "Theorems of Engel and Lie. Semisimple Lie algebras, Cartan's criterion. Universal enveloping algebras, PBW theorem, Serre's construction. Representation theory. Other topics as time permits. Usually offered every second year.\nAn Huang"}

## Cleaning the data
If we want to sort courses by instructor or by code, we need to replace the lists with tuples (which are immutable lists)

In [4]:
for course in courses:
        course['instructor'] = tuple(course['instructor'])
        course['coinstructors'] = tuple([tuple(f) for f in course['coinstructors']])
        course['code']= tuple(course['code'])

In [5]:
print('notice that the instructor and code are tuples now')
courses[1246]

notice that the instructor and code are tuples now


{'limit': 28,
 'times': [{'start': 1080, 'end': 1170, 'days': ['w', 'm']}],
 'enrolled': 4,
 'details': 'Instruction for this course will be offered remotely. Meeting times for this course are listed in the schedule of classes (in ET).',
 'type': 'section',
 'status_text': 'Open',
 'section': '1',
 'waiting': 0,
 'instructor': ('An', 'Huang', 'anhuang@brandeis.edu'),
 'coinstructors': (),
 'code': ('MATH', '223A'),
 'subject': 'MATH',
 'coursenum': '223A',
 'name': 'Lie Algebras: Representation Theory',
 'independent_study': False,
 'term': '1203',
 'description': "Theorems of Engel and Lie. Semisimple Lie algebras, Cartan's criterion. Universal enveloping algebras, PBW theorem, Serre's construction. Representation theory. Other topics as time permits. Usually offered every second year.\nAn Huang"}

# Exploring the data set
Now we will show how to use straight python to explore the data set and answer some interesting questions. Next week we will start learning Pandas/Numpy which are packages that make it easier to explore large dataset efficiently.

Here are some questions we can try to asnwer:
* what are all of the subjects of courses (e.g. COSI, MATH, JAPN, PHIL, ...)
* which terms are represented?
* how many instructors taught at Brandeis last year?
* what were the five largest course sections?
* what were the five largest courses (where we combine sections)?
* which are the five largest subjects measured by number of courses offered?
* which are the five largest courses measured by number of students taught?
* which course had the most sections taught in 20-21?
* who are the top five faculty in terms of number of students taught?
* etc.

a. how many faculty taught COSI courses last year?

In [6]:
cosi_faculty_list = [(c['instructor']) for c in courses if c['subject']=='COSI']
# remove duplicates in cosi_faculty_list using set (b/c set doesn't allow duplicates)
cosi_faculty_list = list(set(cosi_faculty_list))
len(cosi_faculty_list)

27

b. what is the total number of students taking COSI courses last year?

In [7]:
# The total number of students with duplicates (since we only have enrolled num)
sum([(c['enrolled']) for c in courses if c['subject']=='COSI'])

2223

c. what was the median size of a COSI course last year (counting only those
courses with at least 10 students)

In [8]:
import statistics
statistics.median([(c['enrolled']) for c in courses if c['subject']=='COSI' and c['enrolled'] >= 10])

37

d. create a list of tuples (E,S) where S is a subject and E is the number of students
enrolled in courses in that subject, sort it and print the top 10. This shows the top
10 subjects in terms of number of students taught.

In [9]:
subject_list = [(c['subject']) for c in courses]
# remove duplicates in subject_list set (b/c set doesn't allow duplicates)
subject_list = list(dict.fromkeys(subject_list))
enrolled_subject = []
for subject in subject_list:
    enrolled_subject.append((sum([(c['enrolled']) for c in courses if c['subject']==subject]), subject))
# sort the tuples based on the first element (number enrolled) in descending order
enrolled_subject.sort(key=lambda y: y[0],reverse=True)
enrolled_subject[:10] # print the first 10 elements

[(5318, 'HS'),
 (3085, 'BIOL'),
 (2766, 'BUS'),
 (2734, 'HWL'),
 (2322, 'CHEM'),
 (2315, 'ECON'),
 (2223, 'COSI'),
 (1785, 'MATH'),
 (1704, 'PSYC'),
 (1144, 'ANTH')]

e. do the same as in (d) but print the top 10 subjects in terms of number of courses
offered

In [10]:
course_subject = []
for subject in subject_list:
    course_subject.append((len([(c) for c in courses if c['subject']==subject]), subject))
# sort the tuples based on the first element (number enrolled) in descending order
course_subject.sort(key=lambda y: y[0],reverse=True)
course_subject[:10] # print the first 10 elements

[(613, 'BIOL'),
 (498, 'HIST'),
 (417, 'PSYC'),
 (403, 'NEUR'),
 (296, 'BCHM'),
 (288, 'PHYS'),
 (274, 'HS'),
 (272, 'COSI'),
 (266, 'MUS'),
 (265, 'ENG')]

f. do the same as (d) but print the top 10 subjects in terms of number of faculty
teaching courses in that subject

In [11]:
faculty_subject = []
for subject in subject_list:
    faculty_tmp = [(c['instructor']) for c in courses if c['subject']==subject]
    # remove duplicates in faculty_tmp using set (b/c set doesn't allow duplicates)
    faculty_tmp = list(set(faculty_tmp))
    faculty_subject.append((len(faculty_tmp),subject))
# sort the tuples based on the first element (number enrolled) in descending order
faculty_subject.sort(key=lambda y: y[0],reverse=True)
faculty_subject[:10] # print the first 10 elements

[(87, 'HS'),
 (67, 'BIOL'),
 (52, 'ECON'),
 (49, 'BCHM'),
 (47, 'BUS'),
 (47, 'HIST'),
 (46, 'BCBP'),
 (42, 'HWL'),
 (37, 'MATH'),
 (37, 'NEJS')]

g. list the top 20 faculty in terms of number of students they taught

In [12]:
faculty_list = [(c['instructor']) for c in courses]
# remove duplicates in faculty_list using set (b/c set doesn't allow duplicates)
faculty_list = list(set(faculty_list))

faculty_student = []
for faculty in faculty_list:
    faculty_student.append((sum([(c['enrolled']) for c in courses if c['instructor']==faculty]), faculty))
faculty_student.sort(key=lambda y: y[0],reverse=True)
faculty_student[:20] # print the first 10 elements

[(926, ('Leah', 'Berkenwald', 'leahb@brandeis.edu')),
 (583, ('Kene Nathan', 'Piasta', 'kpiasta@brandeis.edu')),
 (515, ('Stephanie', 'Murray', 'murray@brandeis.edu')),
 (489, ('Milos', 'Dolnik', 'dolnik@brandeis.edu')),
 (450, ('Maria', 'de Boef Miara', 'mmiara@brandeis.edu')),
 (439, ('Bryan', 'Ingoglia', 'ingoglia@brandeis.edu')),
 (422, ('Rachel V.E.', 'Woodruff', 'woodruff@brandeis.edu')),
 (411, ('Timothy J', 'Hickey', 'tjhickey@brandeis.edu')),
 (375, ('Daniel', 'Breen', 'dbreen91@brandeis.edu')),
 (365, ('Melissa', 'Kosinski-Collins', 'kosinski@brandeis.edu')),
 (355, ('Claudia', 'Novack', 'novack@brandeis.edu')),
 (342, ('Antonella', 'DiLillo', 'dilant@brandeis.edu')),
 (330, ('Jon', 'Chilingerian', 'chilinge@brandeis.edu')),
 (327, ('Ahmad', 'Namini', 'anamini@brandeis.edu')),
 (316, ('Iraklis', 'Tsekourakis', 'tsekourakis@brandeis.edu')),
 (315, ('Geoffrey', 'Clarke', 'geoffclarke@brandeis.edu')),
 (277, ('Peter', 'Mistark', 'pmistark@brandeis.edu')),
 (275, ('Brenda', 'Ande

h. list the top 20 courses in terms of number of students taking that course (where
you combine different sections and semesters, i.e. just use the subject and
course number)

In [13]:
course_student = []
for course in courses:
    code = course['code']
    if code not in [cs[1] for cs in course_student]:
        students_enrolled = sum([(c['enrolled']) for c in courses if c['code']==code])
        course_student.append((students_enrolled, code))
course_student.sort(key=lambda y: y[0],reverse=True)
course_student[:20] # print the first 10 elements

[(940, ('HWL', '1')),
 (879, ('HWL', '1-PRE')),
 (358, ('BIOL', '14A')),
 (343, ('COSI', '10A')),
 (336, ('PSYC', '10A')),
 (287, ('BIOL', '15B')),
 (280, ('MATH', '10A')),
 (274, ('BIOL', '18B')),
 (262, ('BIOL', '18A')),
 (245, ('CHEM', '29A')),
 (239, ('CHEM', '29B')),
 (236, ('CHEM', '25A')),
 (231, ('PSYC', '51A')),
 (226, ('CHEM', '25B')),
 (225, ('COSI', '12B')),
 (215, ('BUS', '6A')),
 (208, ('CHEM', '18A')),
 (207, ('ECON', '10A')),
 (204, ('MATH', '15A')),
 (201, ('ANTH', '1A'))]

i. Create your own interesting question (each team member creates their own) and
use Python to answer that question.
* Molly's Question: How many independent studies are offered?


In [14]:
len([(c) for c in courses if c['independent_study']==True])

5477

Tony's Question: How many courses of dependent studies are offered?

In [15]:
len([(c) for c in courses if c['independent_study']==False])

2336